In [1]:
#Task 13 : LangGraph
#Generating poem using LangGraph nodes

from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Define the state
class PoemState(TypedDict):
    messages: list[HumanMessage | AIMessage | SystemMessage]

# Initialize Ollama LLM
ollama_llm = OllamaLLM(model="llama3")  # replace "llama2" with your Ollama model

# generate poem
def generate_poem(state: PoemState) -> PoemState:
    prompt_messages = state["messages"]
    # Ollama expects a single string prompt, so join messages
    user_prompt = "\n".join(
        msg.content for msg in prompt_messages if isinstance(msg, (HumanMessage, SystemMessage))
    )
    response = ollama_llm.invoke(user_prompt)  # call Ollama
    return {"messages": [AIMessage(content=response)]}

# Create the graph
graph = StateGraph(PoemState)
graph.add_node("poem_node", generate_poem)
graph.add_edge(START, "poem_node")
graph.add_edge("poem_node", END)
compiled_graph = graph.compile()

# Initial state
initial_state = {
    "messages": [
        SystemMessage(content="You are a creative poet."),
        HumanMessage(content="Write a short beautiful poem about the night sky.")
    ]
}

# Run the graph
result = compiled_graph.invoke(initial_state)

# Print the poem
for msg in result["messages"]:
    if isinstance(msg, AIMessage):
        print(" Generated Poem:\n", msg.content)


 Generated Poem:
 Here's a poem I came up with:

The velvet veil of twilight falls,
As stars awaken, one by one, to call
The celestial show begins anew,
A canvas painted, dark and true.

Like diamonds scattered, free and bright,
Their twinkling whispers echo through the night.
A gentle breeze stirs, a soft sigh,
As constellations twirl, and dance, and fly.

In this great tapestry of endless blue,
Our place is small, yet we are part of it anew.
For in the silence, I hear a voice,
That speaks to hearts, and makes our souls rejoice.

And when I look upon that starry sea,
I feel the mystery, and wonder, set me free.
The night sky whispers secrets, ancient and true,
Reminding us of magic, waiting there for you.
